# GISTDA Wildfire Machine Learning Training

### This project is focused on developing and training machine learning models to predict and monitor wildfires. It utilizes datasets from Sentinel-2 Images as Raster GeoTIFF format which have been wildfire extraction, analyzes environmental factors, and applies machine learning algorithms to enhance prediction performance.

## Import Library Package

##### 1. **Numpy (`numpy`)**
   - **Description**: A powerful library for numerical computations in Python. It provides support for arrays, matrices, and a large collection of mathematical functions to operate on these data structures, making it essential for scientific and statistical analysis.

##### 2. **Pandas (`pandas`)**
   - **Description**: A widely-used library for data manipulation and analysis. It provides data structures such as DataFrames to store and manipulate large datasets, making data cleaning, transformation, and exploration more efficient and intuitive.

##### 3. **Matplotlib (`matplotlib.pyplot`)**
   - **Description**: A plotting library that provides tools for creating a wide range of static, animated, and interactive visualizations in Python. It is commonly used for generating graphs, charts, and other visual data representations.

##### 4. **Seaborn (`seaborn`)**
   - **Description**: A statistical data visualization library built on top of Matplotlib. It offers high-level functions for creating informative and attractive visualizations, especially useful for exploring and understanding data trends and distributions.

##### 5. **OS (`os`)**
   - **Description**: A standard library in Python that provides functions to interact with the operating system, allowing you to work with directories, files, and system paths. It's useful for handling file operations, environment variables, and system commands.

##### 6. **Pickle (`pickle`)**
   - **Description**: A Python module used to serialize and deserialize Python objects, allowing you to save complex data structures to files and load them back into your program. It’s commonly used for saving trained models or intermediate data states.

##### 7. **Dask (`dask.dataframe` and `dask`)**
   - **Description**: A parallel computing library that scales up computations on larger datasets. `dask.dataframe` provides similar functionality to Pandas DataFrames but can handle larger-than-memory data by performing parallel, chunked computations.

##### 8. **Rasterio (`rasterio`)**
   - **Description**: A library for reading and writing geospatial raster data. It’s widely used for working with geospatial data in formats like GeoTIFF, allowing for operations on large image files commonly used in remote sensing and GIS applications.

##### 9. **Scikit-Learn (`sklearn`)**
   - **Description**: A robust library for machine learning that provides simple and efficient tools for data analysis and modeling. It includes modules like `MinMaxScaler` for scaling data, `cross_val_score` and `cross_val_predict` for evaluating models, and various classifiers.

##### 10. **LightGBM (`lightgbm`)**
   - **Description**: A high-performance, gradient-boosting framework developed by Microsoft. It's optimized for speed and efficiency on large datasets and is particularly well-suited for structured data and classification problems.

##### 11. **IPython (`IPython.display`)**
   - **Description**: A library for creating interactive elements in Jupyter Notebooks, such as displaying Markdown, HTML, and other rich content. It’s often used to improve the readability and interactivity of notebook outputs.

##### 12. **Delayed and Compute (`dask.delayed` and `dask.compute`)**
   - **Description**: Functions in the Dask library that allow you to parallelize and execute tasks asynchronously. `delayed` is used to mark a function for lazy evaluation, and `compute` is used to execute the delayed functions, making computations efficient and scalable.

##### 13. **Rasterio Windows (`rasterio.windows`)**
   - **Description**: A submodule in Rasterio that allows for windowed or tiled reading of raster data. This is useful for reading and processing only portions of large raster datasets, improving efficiency when working with large geospatial files.

##### 14. **XGBoost (`xgboost`)**
   - **Description**: An optimized gradient-boosting framework that’s highly effective for predictive modeling tasks, especially classification and regression. It’s known for its speed and performance and is widely used in machine learning competitions for its accuracy.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import dask.dataframe as dd
import rasterio
import dask
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import display, Markdown
from dask import delayed, compute
from rasterio.windows import Window

## Import Raster GeoTIFF files

This code reads large raster files in manageable chunks, processes each chunk, and stores the data in a consolidated DataFrame.

1. **Folder Path and Chunk Size**: Sets the path to the folder containing raster files (`raster_train_file_path`) and defines the chunk size (`CHUNK_SIZE`).

2. **`read_raster_in_chunks` Function**:
   - Reads a raster file chunk-by-chunk.
   - For each chunk, it extracts data from all bands, along with the pixel coordinates.
   - Stores each chunk's data in a temporary DataFrame, which is added to a list.

3. **Task Creation**:
   - Creates a list of tasks using Dask's `@delayed` to process each raster file in parallel.

4. **Compute and Combine**:
   - Computes each task to get individual DataFrames for each file, then combines them into a final DataFrame (`final_df`).

5. **Debug Outputs**:
   - Prints the shape, columns, and a sample of the final DataFrame for inspection.

In [ ]:
# Define the folder containing the raster files
raster_train_file_path = r'Raster_Train'

# Parameters for chunk size
CHUNK_SIZE = 512 # Adjust following your hardware specification

@delayed
def read_raster_in_chunks(raster_path, file, root):
    with rasterio.open(raster_path) as src:
        height, width = src.height, src.width
        num_bands = src.count
        band_names = [f'B{str(i).zfill(2)}' for i in range(1, num_bands + 1)]
        chunk_dfs = []
        
        # Loop over the raster in chunks
        for row in range(0, height, CHUNK_SIZE):
            for col in range(0, width, CHUNK_SIZE):
                window = Window(col_off=col, row_off=row, 
                              width=min(CHUNK_SIZE, width - col),
                              height=min(CHUNK_SIZE, height - row))
                
                # Read all bands at once
                data = src.read(window=window)
                
                # Check if chunk contains any data
                if np.any(data):
                    rows, cols = data[0].shape
                    
                    # Create base DataFrame with coordinates
                    row_coords, col_coords = np.meshgrid(
                        np.arange(row, row + rows),
                        np.arange(col, col + cols),
                        indexing="ij"
                    )
                    
                    chunk_df = pd.DataFrame({
                        'raster_file': file,
                        'subfolder': os.path.basename(root),
                        'x': row_coords.flatten(),
                        'y': col_coords.flatten()
                    })
                    
                    # Add each band's data
                    for band_idx, band_name in enumerate(band_names, 1):
                        chunk_df[band_name] = data[band_idx-1].flatten()
                    
                    chunk_dfs.append(chunk_df)
        
        return pd.concat(chunk_dfs, ignore_index=True) if chunk_dfs else pd.DataFrame()

# Create list of tasks
dask_dfs = [
    read_raster_in_chunks(os.path.join(root, file), file, root)
    for root, dirs, files in os.walk(raster_train_file_path)
    for file in files if file.endswith('.tif')
]

# Compute all tasks
dataframes = compute(*dask_dfs)

# Combine all DataFrames
final_df = pd.concat(dataframes, ignore_index=True)

# Debug prints
print("DataFrame shape:", final_df.shape)
print("\nDataFrame columns:", final_df.columns.tolist())
print("\nSample of data:")
print(final_df.head())

## Exploratory Data Analysis (EDA) & Feature Engineering

This code performs several operations on a DataFrame (`final_df`), converting it to a Dask DataFrame for distributed processing, renaming columns, and dropping unnecessary columns:

1. **Convert to Dask DataFrame**:
   - Converts the existing Pandas DataFrame (`final_df`) into a Dask DataFrame (`ddf`) to enable parallel, distributed processing.

2. **Rename Columns**:
   - Defines a list of new column names for Sentinel-2 bands and other data (`new_col_names`).
   - Renames `ddf` columns using this list to make them more descriptive.

3. **Drop Unneeded Columns**:
   - Drops columns such as `raster_file`, `subfolder`, `x`, `y`, and `dNBR`, keeping only essential information in the DataFrame (`df`).

4. **Display DataFrame**:
   - Uses `display()` to view the DataFrame when needed, triggering Dask's computation.

In [ ]:
# Rename columns
new_col_names = ['raster_file', 'subfolder', 'x', 'y', 'Band_1', 'Band_2', 'Band_3', 'Band_4', 
                 'Band_5', 'Band_6', 'Band_7', 'Band_8', 'Band_8A', 'Band_9', 'Band_11', 
                 'Band_12', 'dNBR', 'NDVI', 'NDWI', 'Burn_Label']
df = final_df.rename(columns=dict(zip(final_df.columns, new_col_names)))

# Drop unnecessary columns
columns_to_drop = ['raster_file', 'subfolder', 'x', 'y', 'dNBR']
df = df.drop(columns=columns_to_drop)

# Display DataFrame (computing results)
print(df.head())  # Displays a small, computed sample

### Check Burn Class

This code checks and displays the counts of burn records in the DataFrame:

1. **Count Burn Labels**:
   - Counts occurrences in the `Burn_Label` column to determine the number of "Burn" and "Unburn" records.
   - Renames the labels: `1` to "Burn" and `0` to "Unburn" for readability.

2. **Display Counts**:
   - Prints the resulting counts to show the distribution of burn and unburned areas.

In [ ]:
# Check Burn Records
burn_counts = df['Burn_Label'].value_counts().rename(index={1: 'Burn', 0: 'Unburn'})

# Display the counts with labels
print(burn_counts)

### Downsampling

This code performs downsampling to balance the dataset by reducing the number of "Unburn" records to match the count of "Burn" records:

1. **Get Burn Count**:
   - Retrieves the count of "Burn" records from `burn_counts`.

2. **Sample Unburned Records**:
   - Selects a random sample of "Unburn" records, equal in size to the number of "Burn" records, using a fixed `random_state` for reproducibility.

3. **Combine Burn and Downsampled Unburn Records**:
   - Combines all "Burn" records with the downsampled "Unburn" sample into a new DataFrame (`downsampled_df`).

4. **Check New Burn Record Counts**:
   - Counts and displays the "Burn" and "Unburn" records in `downsampled_df` to verify balance.

In [ ]:
burn_count = burn_counts['Burn']
unburn_sample = df[df['Burn_Label'] == 0].sample(n=burn_count, random_state=42)

downsampled_df = pd.concat([df[df['Burn_Label'] == 1], unburn_sample])

# Check Burn Records
burn_counts = downsampled_df['Burn_Label'].value_counts().rename(index={1: 'Burn', 0: 'Unburn'})

# Display the counts with labels
print(burn_counts)

## Pre-Processing

### Remove infinite values

This code handles the presence of infinite values and missing data in the `downsampled_df` DataFrame:

1. **Replace Infinite Values**:
   - Replaces both positive and negative infinite values (`np.inf` and `-np.inf`) with `NaN` using `replace()`. This ensures that infinite values do not interfere with further processing.

2. **Drop Rows with Missing Values**:
   - Removes any rows containing `NaN` values using `dropna()`, ensuring the DataFrame only contains valid data.

3. **Display the DataFrame**:
   - Displays the cleaned DataFrame (`downsampled_df`) for inspection.

In [ ]:
# Replacing infinite with nan 
downsampled_df.replace([np.inf, -np.inf], np.nan, inplace=True) 
  
# Dropping all the rows with nan values 
downsampled_df.dropna(inplace=True)

# Printing df 
display(downsampled_df)

### Seperate Burn_Label from DataFrame

This code separates the `Burn_Label` from the main DataFrame and ensures the label is in the correct format:

1. **Separate Burn Label**:
   - Extracts the `Burn_Label` column from `downsampled_df` into a new DataFrame (`burn_label`).

2. **Remove Burn Label from Main DataFrame**:
   - Drops the `Burn_Label` column from `downsampled_df` to ensure only the feature data remains.

3. **Convert Burn Label to Integer**:
   - Changes the data type of the `burn_label` DataFrame to `int32` to ensure consistent and efficient processing.

4. **Display Burn Label**:
   - Displays the modified `burn_label` DataFrame to verify the changes.

In [ ]:
# Seperate Burn_Label from DataFrame
burn_label = downsampled_df[['Burn_Label']]

# Drop Label from DataFrame
downsampled_df = downsampled_df.drop(columns=['Burn_Label'])

# Change type of Label to Integer Format
burn_label = burn_label.astype('int32')
display(burn_label)

### Normalization Data with MinMax Scaler

This code normalizes the feature data and saves the normalization model, while also combining it with the `Burn_Label`:

1. **List Columns**:
   - Creates a list of column names from `downsampled_df` to keep track of the column order after normalization (`cols_norm`).

2. **Normalize the Data**:
   - Imports `MinMaxScaler` from `sklearn` and fits it to the data in `downsampled_df`, which scales the values between 0 and 1.

3. **Save the Scaler**:
   - Saves the fitted `MinMaxScaler` model to a specified path (`MinMax_Scaler.pkl`) for later use.

4. **Apply Normalization**:
   - Normalizes the data by applying the `scaler` to `downsampled_df`, then converts the result back into a DataFrame (`df_norm`) with the original column names.

5. **Check Shape**:
   - Prints the shape of `df_norm` to confirm the normalization was applied correctly.

6. **Concatenate with Burn Label**:
   - Combines the normalized feature data (`df_norm`) with the `burn_label` DataFrame, aligning them by their indices and ensuring the result is a complete dataset.

7. **Display the DataFrame**:
   - Displays the final DataFrame (`df_norm`), which now includes both the normalized features and the `Burn_Label`.


In [ ]:
# Reassign the dataframe with a list of the columns
cols_norm = downsampled_df.columns.tolist()

# Import Normalize technique
scaler = MinMaxScaler()

# Normalize data
scaler.fit(downsampled_df)

# Save the scaler
scaler_save_path = r'Export_Model'
save_path = os.path.join(scaler_save_path, 'DL_MinMax_Scaler.pkl')
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with open(save_path, 'wb') as f:
    pickle.dump(scaler, f)

# Normalize Data
df_norm = scaler.transform(downsampled_df)
df_norm = pd.DataFrame(df_norm, columns=cols_norm)

# Check df_norm shape after normalization
print("Shape of df_norm after normalization:", df_norm.shape)

# Concatenate df_norm with burn_label
df_norm = pd.concat([df_norm.reset_index(drop=True), burn_label.reset_index(drop=True)], axis=1, sort=False)
display(df_norm)

## Neural Network Deployment

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # For progress bar

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming df_norm is defined and contains 'Burn_Label'
X = df_norm.drop(columns=['Burn_Label']).values  # Convert to NumPy array
y = df_norm['Burn_Label'].values  # Convert to NumPy array

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to torch tensors and move them to GPU if available
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32)).to(device)
        self.y = torch.from_numpy(y.astype(np.float32)).unsqueeze(1).to(device)  # Ensure y is 2D for BCELoss
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.len

batch_size = 64

# Instantiate training and test data
train_data = Data(X_train, y_train)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = Data(X_test, y_test)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

import torch
from torch import nn
from torch import optim

input_dim = X_train.shape[1]  # Dynamically get input size
hidden_dim1 = 128
hidden_dim2 = 64
output_dim = 1  # Binary classification

class DeepNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(DeepNeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim1)
        self.bn1 = nn.BatchNorm1d(hidden_dim1)
        self.layer_2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.bn2 = nn.BatchNorm1d(hidden_dim2)
        self.layer_3 = nn.Linear(hidden_dim2, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.bn1(self.layer_1(x)))
        x = torch.relu(self.bn2(self.layer_2(x)))
        x = torch.sigmoid(self.layer_3(x))  # Binary classification output
        return x

# Move model to GPU
model = DeepNeuralNetwork(input_dim, hidden_dim1, hidden_dim2, output_dim).to(device)
print(model)

In [ ]:
# Define loss function and optimizer
learning_rate = 0.1
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training Loop with Progress Display
num_epochs = 50
loss_values = []

print("Starting Training...")

for epoch in range(num_epochs):
    epoch_loss = 0.0
    with tqdm(train_dataloader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")

        for X, y in tepoch:
            optimizer.zero_grad()  # Reset gradients
            
            pred = model(X)  # Forward pass
            loss = loss_fn(pred, y)  # Compute loss
            
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            epoch_loss += loss.item()
            tepoch.set_postfix(loss=loss.item())  # Show loss in progress bar

    avg_epoch_loss = epoch_loss / len(train_dataloader)
    loss_values.append(avg_epoch_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_epoch_loss:.4f}")

print("Training Complete ✅")

In [ ]:
step = range(len(loss_values))

fig, ax = plt.subplots(figsize=(8,5))
plt.plot(step, np.array(loss_values))
plt.title("Step-wise Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
import itertools  # Import this at the top of your script

# Initialize required variables
y_pred = []
y_test = []
correct = 0
total = 0

with torch.no_grad():
    for X, y in test_dataloader:
        outputs = model(X)  # Get model outputs
        predicted = np.where(outputs.numpy() < 0.5, 0, 1)  # Convert to NumPy and apply threshold
        predicted = list(itertools.chain(*predicted))  # Flatten predictions
        y_pred.append(predicted)  # Append predictions
        y_test.append(y.numpy())  # Append true labels as NumPy
        total += y.size(0)  # Increment total count
        correct += (predicted == y.numpy()).sum().item()  # Count correct predictions

print(f'Accuracy of the network on the 3300 test instances: {100 * correct // total}%')

y_pred = list(itertools.chain(*y_pred))
y_test = list(itertools.chain(*y_test))

report = classification_report(y_test, y_pred, output_dict=True)
cm = confusion_matrix(y_test, y_pred)

model_result = [{
    'Classifier': 'Neural Network',
    'Class 0 - Precision': report['0']['precision'],
    'Class 0 - Recall': report['0']['recall'],
    'Class 0 - F1-Score': report['0']['f1-score'],
    'Class 1 - Precision': report['1']['precision'],
    'Class 1 - Recall': report['1']['recall'],
    'Class 1 - F1-Score': report['1']['f1-score'],
    'Average - Precision': report['macro avg']['precision'],
    'Average - Recall': report['macro avg']['recall'],
    'Average - F1-Score': report['macro avg']['f1-score'],
    'Accuracy': report['accuracy'],
    'Confusion Matrix': cm
}]

model_result_df = pd.DataFrame(model_result)

display(Markdown("### Classification Report of Neural Network"))
display(model_result_df)
    
# Plot Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix - Neural Network')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()